In [ ]:
!pip install git+https://github.com/huggingface/transformers@main
!pip install accelerate
!pip install git+https://github.com/AJStangl/gpt-model-finetuning@master

In [ ]:
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import gc
import os
import logging
import pandas
import torch
import gc
from src.datasets.reddit_dataset import RedditDataset
import pandas
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import logging

In [ ]:
parent_directory = "/content/drive/MyDrive/RawData"

out_dir = f"{parent_directory}/mega_pablo_bot"

tokenizer_path = f"{out_dir}"

data_path = f"{out_dir}/filtered_all.txt"


check_point_name = 'checkpoint-3000'

continue_path = f"{out_dir}/{check_point_name}"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(out_dir)
model = GPT2LMHeadModel.from_pretrained(continue_path).cuda()

In [ ]:
from src.datasets.reddit_dataset import RedditDataset

generator = torch.Generator()

generator.manual_seed(0)

valid_lines = open(data_path, 'r', encoding='utf-8').readlines()

logging.info(f":: Total Number Of Samples {len(valid_lines)}")

max_length = max([len(tokenizer.encode(prompt)) for prompt in valid_lines])

logging.info(f":: Max Length Of Sample {max_length}")

dataset = RedditDataset(valid_lines, tokenizer, max_length=max_length)

train_size = int(0.9 * len(dataset))

train_dataset, eval_dataset = random_split(dataset, [train_size, len(dataset) - train_size], generator=generator)

In [ ]:
training_args = TrainingArguments(output_dir=out_dir)
training_args.num_train_epochs = 5
training_args.logging_steps=100
training_args.save_steps=1000
training_args.weight_decay=0.05
training_args.logging_dir='./logs'
training_args.fp16=True
training_args.auto_find_batch_size=True
training_args.gradient_accumulation_steps=50
training_args.learning_rate=1e-4

In [ ]:
Trainer(model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[0] for f in data])}).train(continue_path)